In [9]:
import tushare as ts
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

data_clean = pd.read_csv("future_ss2312_tick.csv")


# 1. 数据加载和预处理
# data_clean = data.sort_values(by='trade_time')
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')
data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])
data_clean['volume'] = pd.to_numeric(data_clean['数量'], errors='coerce')


# 2. 特征生成
# Example usage:
rolling_windows = {'mean': 300, 'std': 300, 'rsi': 400}
ewm_spans = {'short': 200, 'long': 800, 'signal': 800}

# 滚动统计特征
data_clean['rolling_mean'] = data_clean['close'].rolling(window=rolling_windows['mean']).mean()
data_clean['rolling_std'] = data_clean['close'].rolling(window=rolling_windows['std']).std()


# Calculate RSI
delta = data_clean['close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=rolling_windows['rsi']).mean()
avg_loss = loss.rolling(window=rolling_windows['rsi']).mean()
rs = avg_gain / avg_loss
data_clean['RSI'] = 100 - (100 / (1 + rs))

# Calculate MACD
short_ema = data_clean['close'].ewm(span=ewm_spans['short'], adjust=False).mean()
long_ema = data_clean['close'].ewm(span=ewm_spans['long'], adjust=False).mean()
data_clean['MACD'] = short_ema - long_ema
data_clean['MACD_signal'] = data_clean['MACD'].ewm(span=ewm_spans['signal'], adjust=False).mean()

# Shift RSI and MACD to use them as features for next timestep
data_clean['RSI_shifted'] = data_clean['RSI'].shift(1)
data_clean['MACD_shifted'] = data_clean['MACD'].shift(1)
data_clean['MACD_signal_shifted'] = data_clean['MACD_signal'].shift(1)

# 累积成交量特征
# 计算单位时间内的成交量变化
data_clean['volume_change'] = data_clean['volume'].diff()

# 累积成交量特征生成
data_clean['volume_rolling_mean'] = data_clean['volume'].rolling(window=300).mean()


# 价格变化率和成交量变化
data_clean['price_change_rate'] = data_clean['close'].pct_change()

# 定义标签
data_clean['label'] = (data_clean['close'].shift(-1000) > data_clean['close']).astype(int)


# 3. 分割数据
# 分割数据为训练集和测试集
# Convert the 'trade_time' column to datetime
data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# 将时间分解为小时和分钟
data_clean['hour'] = data_clean['trade_time'].dt.hour
data_clean['minute'] = data_clean['trade_time'].dt.minute

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-09-01 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-10 09:00:00')]

test_data = data_clean[(data_clean['trade_time'] >= '2023-10-10 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-31 09:00:00')]


features = ['rolling_mean', 'rolling_std', 'RSI', 'MACD', 'volume_rolling_mean', 'price_change_rate', 'volume_change','hour','minute']


X_train = train_data[features].dropna()
y_train = train_data['label'][X_train.index]
X_test = test_data[features].dropna()
y_test = test_data['label'][X_test.index]

# # 初始化标准化缩放器
# scaler = StandardScaler()

# # 使用训练数据拟合缩放器，然后转换训练数据
# X_train_scaled = scaler.fit_transform(X_train)

# 继续使用缩放后的数据进行模型训练
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [8]:
def predict_next_move(tick, model, rolling_windows, ewm_spans, historical_data,scaler):
    
    # 将新的 tick 数据追加到历史数据中
    historical_data = pd.concat([historical_data, pd.DataFrame([tick])], ignore_index=True)
    
    
    # 检查我们是否有足够的数据来计算滚动和EWM特征
    if len(historical_data) >= max(rolling_windows['mean'], rolling_windows['std'], rolling_windows['rsi'], ewm_spans['long']):
        # 在历史数据上计算滚动平均和标准差
        historical_data['rolling_mean'] = historical_data['close'].rolling(window=rolling_windows['mean'], min_periods=1).mean()
        historical_data['rolling_std'] = historical_data['close'].rolling(window=rolling_windows['std'], min_periods=1).std()

        # 在历史数据上计算RSI
        delta = historical_data['close'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        avg_loss = loss.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        rs = avg_gain / avg_loss.replace(0, 1)  # 避免除以零
        historical_data['RSI'] = 100 - (100 / (1 + rs))

        # 在历史数据上计算MACD
        short_ema = historical_data['close'].ewm(span=ewm_spans['short'], adjust=False, min_periods=1).mean()
        long_ema = historical_data['close'].ewm(span=ewm_spans['long'], adjust=False, min_periods=1).mean()
        historical_data['MACD'] = short_ema - long_ema
        historical_data['MACD_signal'] = historical_data['MACD'].ewm(span=ewm_spans['signal'], adjust=False, min_periods=1).mean()

        # 将RSI和MACD移位以用作下一个时间步的特征
        historical_data['RSI_shifted'] = historical_data['RSI'].shift(1)
        historical_data['MACD_shifted'] = historical_data['MACD'].shift(1)
        historical_data['MACD_signal_shifted'] = historical_data['MACD_signal'].shift(1)
        
        #累积成交量
        historical_data['volume_rolling_mean'] = historical_data['volume'].rolling(window=300).mean()

        # 价格变化率和成交量变化
        historical_data['price_change_rate'] = historical_data['close'].pct_change()
        historical_data['volume_change'] = historical_data['volume'].diff()

        # 使用最后一行数据进行预测
        X_new = historical_data.iloc[-1:][features]
        
        # 检查X_new是否包含NaN值
        if X_new.isnull().values.any():
            # 处理包含NaN值的行（例如，跳过预测或使用占位符值）
            # 例如，我们可以返回None或一个特定的信号表示数据不足
            return None, historical_data
        else:
            # # 标准化X_new
            # X_new_scaled = scaler.transform(X_new)
            # # 进行预测
            # prediction_proba = model.predict_proba(X_new_scaled)
            # 进行预测
            prediction_proba = model.predict_proba(X_new)
            # 获取预测为类别1的概率
            probability_of_one = prediction_proba[0][1]
            return probability_of_one, historical_data
    else:
        # 数据不足以进行预测
        return None, historical_data




# Initialize historical_data with the correct column names and types if necessary
historical_data = pd.DataFrame()

initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.9
sold_threshold = 0.6
transactions = []
minute_count = 0



for idx, row in test_data.iterrows():
    current_probability, historical_data = predict_next_move(row, model, rolling_windows, ewm_spans, historical_data, scaler)
    # print(current_probability)
    # print(row['trade_time'])
    # print(row['close'])
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 1000 and stock_quantity > 0 and current_probability<sold_threshold:
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0
        
    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_tick.csv')

0.91
Time: 2023-10-10 09:52:01 - Action: BUY at 14665.0, Quantity: 6.0,Funds:11998.0
0.52
Time: 2023-10-10 10:12:38 - Action: SELL at 14650.0, Quantity: 6.0,Funds:99898.0, Price Change: 跌
0.92
Time: 2023-10-10 10:30:28 - Action: BUY at 14630.0, Quantity: 6.0,Funds:12106.0
0.56
Time: 2023-10-10 10:41:55 - Action: SELL at 14635.0, Quantity: 6.0,Funds:99916.0, Price Change: 涨
0.91
Time: 2023-10-10 10:49:21 - Action: BUY at 14635.0, Quantity: 6.0,Funds:12094.0
0.59
Time: 2023-10-10 11:00:36 - Action: SELL at 14620.0, Quantity: 6.0,Funds:99814.0, Price Change: 跌
0.91
Time: 2023-10-10 11:04:20 - Action: BUY at 14625.0, Quantity: 6.0,Funds:12052.0
0.59
Time: 2023-10-10 11:17:48 - Action: SELL at 14620.0, Quantity: 6.0,Funds:99772.0, Price Change: 跌
0.91
Time: 2023-10-10 11:21:13 - Action: BUY at 14620.0, Quantity: 6.0,Funds:12040.0
0.54
Time: 2023-10-10 13:35:16 - Action: SELL at 14620.0, Quantity: 6.0,Funds:99760.0, Price Change: 平
0.94
Time: 2023-10-10 13:39:23 - Action: BUY at 14630.0, Qua

KeyboardInterrupt: 